### Land cover areas per elevation ranges with the 3D Wetlands datasets in GEE

by: Luis Lizcano-Sandoval<br/>
updated: 11/11/2021

In [2]:
## Import libraries:
import ee
import numpy as np
from IPython.display import display, Image
#from pprint import pprint

ee.Initialize()
print('EE API version: ',ee.__version__)

EE API version:  0.1.257


### Import collections

In [2]:
## Counties
regions = ee.FeatureCollection('users/lizcanosandoval/wv-regions/wv_counties_buffer')

## Import DEM collection:
dem = ee.ImageCollection("projects/imars-3d-wetlands/DEM_GoM")

## Import image collection:
wetlands = ee.ImageCollection("projects/imars-3d-wetlands/alabama")\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/fl_big_bend"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/fl_monroe"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/fl_ne"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/fl_panhandle"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/fl_se"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/fl_sw"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/louisiana_e"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/louisiana_w"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/mississippi"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/texas_central"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/texas_ne"))\
  .merge(ee.ImageCollection("projects/imars-3d-wetlands/texas_sw"))

### Check 'regions' properties and prepare metadata

In [3]:
## Check State names
print(regions.aggregate_array('1_State').distinct().getInfo())

['Alabama', 'Florida', 'Georgia', 'Louisiana', 'Mississippi', 'Texas', 'Tamaulipas']


In [3]:
## Check County names
stateLabel = 'Florida'
state = regions.filterMetadata('1_State','equals',stateLabel)
print(state.aggregate_array('2_County').distinct().sort().getInfo())

['Alachua', 'Bay', 'Bradford', 'Brevard', 'Broward', 'Calhoun', 'Charlotte', 'Citrus', 'Clay', 'Collier', 'Columbia', 'Desoto', 'Dixie', 'Duval', 'Escambia', 'Flagler', 'Franklin', 'Gadsden', 'Gilchrist', 'Glades', 'Gulf', 'Hamilton', 'Hardee', 'Hendry', 'Hernando', 'Highlands', 'Hillsborough', 'Holmes', 'Indian River', 'Jackson', 'Jefferson', 'Lafayette', 'Lake', 'Lee', 'Leon', 'Levy', 'Liberty', 'Madison', 'Manatee', 'Marion', 'Martin', 'Miami-Dade', 'Monroe', 'Nassau', 'Okaloosa', 'Okeechobee', 'Orange', 'Osceola', 'Palm Beach', 'Pasco', 'Pinellas', 'Polk', 'Putnam', 'Saint Johns', 'Saint Lucie', 'Santa Rosa', 'Sarasota', 'Seminole', 'Sumter', 'Suwannee', 'Taylor', 'Union', 'Volusia', 'Wakulla', 'Walton', 'Washington']


In [4]:
## Select county, and set time and end months to filter collection
countyLabel = 'Pinellas'
county = state.filterMetadata('2_County','equals',countyLabel)
## Select periods of months and years of interest.
mo_start = 7
mo_end = 12
yr_start = 2014
yr_end = 2018

In [3]:
## Set elevation ranges at x steps
minRange = 0.0
maxRange = 3.0
step = 0.1

# Create list of numbers
demRanges = np.arange(minRange,maxRange+step,step).tolist()
# Reformat numbers
demRanges = [round(num,1) for num in demRanges]
print(demRanges)

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0]


In [6]:
## Filter collection
filterCollection = wetlands.filterBounds(county) \
    .filterMetadata('system:index','not_contains','Cloudy') \
    .filter(ee.Filter.calendarRange(mo_start,mo_end,'month'))
    #.filter(ee.Filter.calendarRange(yr_start,yr_end,'year'))

print(filterCollection.size().getInfo())

79


### Clean misclassified pixels

In [7]:
## Function to select pixels equal to 11-developed
def developed(image):
    mask = ee.Image(image).eq(11)
    return image.updateMask(mask)

## Function to create mask of water only
def water(image):
    mask = ee.Image(image).eq(3)
    return image.updateMask(mask)

## Function to clean missclassified pixels
def cleanDeveloped(collection):
    # Clean misclassified cloudy pixels:
    #fullCollection = collection.filterBounds(county)
    # Select only developed pixels
    devCollection = collection.map(developed)
    # Create mosaic suming pixel values
    sumDeveloped = devCollection.sum()
    # Select pixels greater than 33 (presence of developed class in 3 or more images)
    selectThreshold = sumDeveloped.gt(32)
    # Create a mask for misclassified pixels
    justDeveloped = sumDeveloped.updateMask(selectThreshold)
    # Some areas over water bodies have cloudy pixels we can mask out
    waterCollection = collection.map(water)
    waterMask = ee.ImageCollection(waterCollection).max().mask().Not()
    return justDeveloped.updateMask(waterMask)

## Apply water mask on the cleaned collection. This is the mask for "real" developed pixels.
realDeveloped = cleanDeveloped(filterCollection)
print(realDeveloped.getInfo())

{'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -9.223372036854776e+18, 'max': 9.223372036854776e+18}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


### Final mosaic

In [8]:
## Function to clean mosaic
def clean_function(image):
    developed = image.eq(11) #Developed
    imageUpdated = image.updateMask(developed) #Developed mask
    falseImg = realDeveloped.mask().Not()
    maskDev = imageUpdated.updateMask(falseImg).mask().Not()
    imageMask = image.updateMask(maskDev)
    return imageMask

cleanCollection = filterCollection.map(clean_function)

## Mosaic
mosaicImage = ee.ImageCollection(cleanCollection).max().clip(county)

### Visualize mosaic

In [ ]:
## Palette color codes
palette = [
"#8C8C8C",       # 0 no data
"#FFFFFF",       # 1 cloud
"#621E01",       # 2 soil
"#1F59CC",       # 3 water
"FF00FF",        # 4 - unused
"#E2D924",       # 5 dead grass
"#7CCD11",       # 6 marsh
"#AD6309",       # 7 scrub
"#27D310",       # 8 grass
"106F00",        # 9 forested upland
"1EE682",        # 10 forested wetland
"1E1E1E",        # 11 developed
]

#### comment to show mosaic:
# mosaicMap = Image(url=mosaicImage.getThumbUrl({
#     'dimensions': '500',
#     'min':0,
#     'max':11,
#     'palette':palette,
#     'region': ee.Geometry(county.geometry(10)).bounds()
#     }))
# display(mosaicMap)

### Clip DEM to county/area

In [9]:
## Clip DEM by counties selected:
## Filter the collection by dates and counties selected.
filterDEM = dem.filterBounds(county);
clipDEM = filterDEM.mosaic().clip(county);

### Define function to calculate area

In [10]:
## Geometry used for exporting arguments
geometry = ee.Geometry(county.geometry(10)).bounds()

## Function to calculate areas
def calculateAreas(mosaic):
    areaImage = ee.Image.pixelArea().addBands(mosaic)
    
    ## Get total area (m2) per class:
    areas = areaImage.reduceRegion(
        reducer= ee.Reducer.sum().group(
            groupField= 1,
            groupName= 'code'),
        geometry= geometry,
        scale= 2,
        maxPixels= 1e13
        )
    ## ee.List:
    classes = [{'class':'Cloud'},{'class':'Soil'},{'class':'Water'},{'class':'Dead grass'},{'class':'Marsh'},{'class':'Scrub'},
              {'class':'Grass'},{'class':'Forested upland'},{'class':'Forested wetland'},{'class':'Developed'}]
    classAreas = (ee.List(areas.get('groups')).zip(classes))
    
    ## Convert m^2 to km^2:
    def area_list(item):
        #areaDict = ee.Dictionary(item)
        dict1 = ee.Dictionary(ee.List(item).get(0))
        dict2 = ee.Dictionary(ee.List(item).get(1))
        areaDict = dict1.combine(dict2)
        CODE = ee.Number(areaDict.get('code')).format()
        AREA = ee.Number(areaDict.get('sum')).divide(1e6)
        CLASS = areaDict.get('class') ##cannot add a class label when respective code is missing
        
        feature = ee.Feature(None,{'state':stateLabel,'county':countyLabel,'code':CODE,'class':CLASS,'area_km2':AREA})
        return feature
    
    featureCollection = ee.FeatureCollection(classAreas.map(area_list))
    return featureCollection

### Change pixel values

In [11]:
## Reproject mosaic and remap pixel values
fr = [0,1,2,3,5,6,7,8,9,10,11]
to = [0,1,2,3,4,5,6,7,8,9,10]
mosaic_repr = mosaicImage.reproject(
    crs= 'EPSG:4326',
    scale= 2).remap(fr,to)

### Export files to GDrive

In [12]:
## Export areas per elevation ranges
for i in list(range(len(demRanges)))[0:-1]:
    minElev = demRanges[i]
    maxElev = demRanges[i+1]
    
    # Function to mask ranges
    def maskRange(image,min,max):
        mask = ee.Image(image).gte(min).And(ee.Image(image).lt(max))
        return image.updateMask(mask)
    
    # Get DEM and Mosaic ranges
    demRange = maskRange(clipDEM,minElev,maxElev).mask()
    mosaicRange = mosaic_repr.updateMask(demRange)
    
    # Estimate areas
    areasFile = calculateAreas(mosaicRange)
    
    # File name
    fileName = stateLabel+'_'+countyLabel+'_'+str(minElev)+'-'+str(maxElev)+'_m'

    ## export
    task = ee.batch.Export.table.toDrive(\
          collection= areasFile,
          description= fileName,
          folder= 'Wetlands',
          fileNamePrefix= fileName,
          fileFormat= 'CSV')

    task.start()
    print('Submitted '+fileName)
print('Export completed')

Submitted Florida_Pinellas_1.6-1.7_m
Submitted Florida_Pinellas_1.7-1.8_m
Submitted Florida_Pinellas_1.8-1.9_m
Submitted Florida_Pinellas_1.9-2.0_m
Submitted Florida_Pinellas_2.0-2.1_m
Submitted Florida_Pinellas_2.1-2.2_m
Submitted Florida_Pinellas_2.2-2.3_m
Submitted Florida_Pinellas_2.3-2.4_m
Submitted Florida_Pinellas_2.4-2.5_m
Submitted Florida_Pinellas_2.5-2.6_m
Submitted Florida_Pinellas_2.6-2.7_m
Submitted Florida_Pinellas_2.7-2.8_m
Submitted Florida_Pinellas_2.8-2.9_m
Export completed
